In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/MyDrive/2021_2학기/기계학습

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data = pd.read_csv('./data/train.csv', encoding = 'utf-8')
test_data = pd.read_csv('./data/test.csv', encoding = 'utf-8')
train_data.drop_duplicates(subset=['mail'], inplace=True)

label_data = train_data['label']
mail_data = train_data['mail']
mail_test = test_data['mail']
mail_train, mail_cv, label_train, label_cv = train_test_split(mail_data, label_data, test_size=0.2, random_state=0, stratify=label_data)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(mail_train)
mail_train_encoded = tokenizer.texts_to_sequences(mail_train)
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1
max_len = max(len(l) for l in mail_train_encoded)
mail_train_padded = pad_sequences(mail_train_encoded, maxlen = max_len)

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(mail_cv)
mail_cv_encoded = tokenizer.texts_to_sequences(mail_cv)
mail_cv_padded = pad_sequences(mail_cv_encoded, maxlen = max_len)

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(mail_test)
mail_test_encoded = tokenizer.texts_to_sequences(mail_test)
max_len_test = max(len(l) for l in mail_test_encoded)
mail_test_padded = pad_sequences(mail_test_encoded, maxlen = max_len_test)


In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model = Sequential()
model.add(Embedding(vocab_size, 32))
# model.add(SimpleRNN(32))
# model.add(LSTM(32))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(1, activation='softmax'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc', f1_m, precision_m, recall_m])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_f1_m', mode='max', verbose=1, save_best_only=True)
history = model.fit(mail_train_padded, label_train, epochs=10, batch_size=64, callbacks=[es, mc], validation_split=0.23)

loss, accuracy, f1_score, precision, recall = model.evaluate(mail_cv_padded, label_cv, batch_size=64, verbose=0)
print('loss: ', loss)
# print(loss)
print('accuracy: ', accuracy)
# print(accuracy)
print('f1_score: ', f1_score)
# print(f1_score)
print('precision: ', precision)
# print(precision)
print('recall: ', recall)
# print(recall)

prediction = model.predict(mail_test_padded, batch_size=64)
for idx, p in enumerate(prediction):
    if p > 0.5 :
        prediction[idx] = bool(1)
    else :
        prediction[idx] = bool(0)

submission = pd.read_csv('./result.csv', encoding = 'utf-8')
submission['label'] = prediction
submission[['label']]=submission[['label']].astype(int)
submission.to_csv('./submissionRnn2.csv', index=False)
# submission.info()

epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model acc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
